In [1]:
import numpy as np

def sigmoid(z):
    return 1 / (1 + np.exp(-z))

def logistic_sigmoid_regression(Xbar, y, w_init, eta, tol = 1e-4, max_count = 10000):
    w = [w_init]
    it = 0
    N = Xbar.shape[0]
    d = Xbar.shape[1]
    count = 0
    check_w_after = 20
    while count < max_count:
        # Mix data
        mix_id = np.random.permutation(N)
        for i in mix_id:
            xi = Xbar[i]
            yi = y[i]
            zi = sigmoid(np.dot(xi, w[-1]))
            w_new = w[-1] + eta*(yi - zi)*xi
            count += 1
            # stopping criteria
            if count%check_w_after == 0:
                if np.linalg.norm(w_new - w[-check_w_after]) < tol:
                    return w
            w.append(w_new)
    return w

In [2]:
from pandas import read_csv

# Đọc dữ liệu từ file CSV
data = read_csv("Admission_Predict.csv")

# Lấy dữ liệu vào các biến
gre = data['GRE Score'].tolist()
tfl = data['TOEFL Score'].tolist()
unirt = data['University Rating'].tolist()
sop = data['SOP'].tolist()
lor1 = data['LOR '].tolist()
cgpa1 = data['CGPA'].tolist()
research_exp = data['Research'].tolist()
prob_Admit = data['Chance of Admit'].tolist()

#i, ii
# Chọn 350 dòng đầu làm dữ liệu training, phần còn lại là test
X_train = np.array([gre[:350], tfl[:350], unirt[:350], sop[:350], lor1[:350], cgpa1[:350], research_exp[:350]]).T
X_test = np.array([gre[350:], tfl[350:], unirt[350:], sop[350:], lor1[350:], cgpa1[350:], research_exp[350:]]).T
#iii
# Tạo cột X0 ≡ 1
Xbar_train = np.concatenate((np.ones((X_train.shape[0], 1)), X_train), axis=1)
Xbar_test = np.concatenate((np.ones((X_test.shape[0], 1)), X_test), axis=1)

# Định nghĩa nhãn cho training data và test data
Y_train = np.array([1 if x >= 0.75 else 0 for x in prob_Admit[:350]])
Y_test = np.array([1 if x >= 0.75 else 0 for x in prob_Admit[350:]])

C:\Users\chuho\AppData\Local\Temp\ipykernel_4312\662784148.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  from pandas import read_csv


In [3]:
# Định nghĩa các tham số ban đầu
w_init = np.random.randn(Xbar_train.shape[1])
eta = 0.05
#iv
# Thực hiện hồi quy logistic
w = logistic_sigmoid_regression(Xbar_train, Y_train, w_init, eta)
#v
# In các hệ số kết quả ra màn hình
print("Các hệ số của mô hình:")
print(w[-1])

Các hệ số của mô hình:
[ -6.62244844 -90.81975049 211.22934246 143.13331619 111.33687068
  93.42590089  45.86140068  54.78215742]


C:\Users\chuho\AppData\Local\Temp\ipykernel_4312\736489293.py:4: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))


In [4]:
#vi
def predict(X, w):
    """
    Predict label for each sample in X
    """
    N = X.shape[0]
    preds = []
    for i in range(N):
        pred = sigmoid(np.dot(X[i], w))
        preds.append(pred)
    return preds

# Dự đoán cho dữ liệu test
Y_pred = predict(Xbar_test, w[-1])

# Chuyển đổi dự đoán thành nhãn 0 hoặc 1 dựa trên ngưỡng 0.5
Y_pred_labels = np.array([1 if y >= 0.5 else 0 for y in Y_pred])

# Đối chiếu kết quả với nhãn thực tế
accuracy = np.mean(Y_pred_labels == Y_test)
precision = np.sum((Y_pred_labels == 1) & (Y_test == 1)) / np.sum(Y_pred_labels == 1)
recall = np.sum((Y_pred_labels == 1) & (Y_test == 1)) / np.sum(Y_test == 1)

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)

Accuracy: 0.56
Precision: nan
Recall: 0.0


C:\Users\chuho\AppData\Local\Temp\ipykernel_4312\736489293.py:4: RuntimeWarning: overflow encountered in exp
  return 1 / (1 + np.exp(-z))
C:\Users\chuho\AppData\Local\Temp\ipykernel_4312\1475241169.py:20: RuntimeWarning: invalid value encountered in scalar divide
  precision = np.sum((Y_pred_labels == 1) & (Y_test == 1)) / np.sum(Y_pred_labels == 1)


In [5]:
#b
# Sử dụng thư viện scikit-learn để hồi quy tuyến tính
from sklearn.linear_model import LinearRegression

# Tạo mô hình hồi quy tuyến tính
lin_reg = LinearRegression()

# Huấn luyện mô hình trên tập training
lin_reg.fit(X_train, prob_Admit[:350])

# Dự đoán kết quả cho tập test
Y_pred_linear = lin_reg.predict(X_test)

# Tính trung bình bình phương sai số
mse = np.mean((Y_pred_linear - prob_Admit[350:]) ** 2)
print("Mean Squared Error:", mse)

Mean Squared Error: 0.004641678440367148


In [6]:
#c
# Sử dụng thư viện scikit-learn để Naive Bayes
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score, precision_score, recall_score

# Tạo mô hình Naive Bayes
nb_model = GaussianNB()

# Huấn luyện mô hình trên tập training
nb_model.fit(X_train, Y_train)

# Dự đoán kết quả cho tập test
Y_pred_nb = nb_model.predict(X_test)

# Tính độ chính xác, precision và recall
accuracy_nb = accuracy_score(Y_test, Y_pred_nb)
precision_nb = precision_score(Y_test, Y_pred_nb)
recall_nb = recall_score(Y_test, Y_pred_nb)

print("Naive Bayes Accuracy:", accuracy_nb)
print("Naive Bayes Precision:", precision_nb)
print("Naive Bayes Recall:", recall_nb)

Naive Bayes Accuracy: 0.9
Naive Bayes Precision: 0.9473684210526315
Naive Bayes Recall: 0.8181818181818182
